## CHEME 5660: The Price of Treasury Bill, Notes and Bonds

#### Treasury Bills
Treasury bills (called T-bills) are short-term debt instruments ranging from a few days to 52 weeks. When a T-bill matures, the bill holder is paid the par amount. The U.S. Treasury sells T-bills at a discount from the par amount; rarely are they sold at a price equal to the par amount. The U.S. Treasury auctions T-bills with terms of 4, 8, 13, 26, and 52 weeks on a regular schedule. 

T-bills are an example of a zero-coupon fixed-income instrument; thus, T-bills do not make coupon payments during the bond term. Instead, the bondholder receives the par value at the bill's period. 

Let a zero-coupon T-bill have a term of $T$-years with an annual market interest rate of $\bar{r}$. Then, the _fair price_ for a zero-coupon T-bill, denoted by $V_{B}$, is the future par value $V_{P}$ of the bill discounted to the time of the purchase:

$$V_{B} = \frac{V_{P}}{\left(1+\bar{r}\right)^{T}}$$

#### Treasury Bonds
Treasury Bonds are long-term U.S Treasury debt instruments. Treasury bonds pay a fixed rate of interest (the coupon rate) every six months until the bond's maturity. The U.S. Treasury issues bonds with terms of 20 or 30 years. When a bond matures, the bondholder receives the face value of the bond. Bonds can be held until maturity or sold before maturity. 

When a U.S. Treasury bond matures, the U.S government repays the debt by paying the bond's par value. The bond's coupon rate determines the interest payment: the annual payment is the coupon rate times the bond's par value. The coupon rate, maturity date, and par value of the bond are part of the contract between the issuer, the U.S. government, and the bondholder (you).

Let the term of a bond be $T$-years, with an annual coupon rate of $\bar{c}$. Then, a _fair price_ for the bond, denoted by $V_{B}$, is the present value of coupon payments $C$ plus the discounted par value $V_{P}$ of the bond:


$$V_{B} = \frac{V_{P}}{\left(1+i\right)^{N}}+\sum_{j=1}^{N}\frac{C}{\left(1+i\right)^{j}}$$

where $N$ denotes the number of coupon payments, $i = \bar{r}/2$ denotes the market interest rate, and $C=(\bar{c}/2)\cdot{V_{P}}$ is the value of the coupon payment.

### Lab setup
The code block below installs (and loads) any [Julia](https://julialang.org) packages that we need to complete the calculations. 

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-1-treasury-debt-instruments`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-1-treasury-debt-instruments/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Markets-Mayhem-Example-Notebooks/labs/lab-1-treasury-debt-instruments/Manifest.toml`


In [2]:
# load external packages -
using CSV
using DataFrames
using Plots

### Load the lab 1 code library
The call to the `include` function loads the `CHEME-5660-Lab-1-Library.jl` library into the notebook; the library contains types and functions we use during the lab:

* The `ZeroCouponBillModel` and `MultipleCouponBondModel` types encapsulate configuration information for a zero coupon debt security, e.g., a treasury note, and multi-coupon debt security, e.g., a treasury bond. 
* The `price` function takes a configuration model and solves for the debt security price at the time the security is issued.

In [3]:
# load the code library -
include("CHEME-5660-Lab-1-Library.jl");

### Control simulations (check if our implementation is correct)

#### a) Code Check: Zero-coupon Treasury Bill Price

Let's check the implementation of the ``price`` function for a zero coupon treasury bill. Before we do anything, let's test the price function answer against examples we find online; one example is in the lecture notes from [IEOR 4700 at Columbia by Prof. K. Sigman](http://www.columbia.edu/~ks20/FE-Notes/FE-Notes-Sigman.html).

In [4]:
# build a zero coupon model for the test calculation -
zero_coupon_test_model = ZeroCouponBillModel()
zero_coupon_test_model.r̄ = 0.04;      # annual market interest rate
zero_coupon_test_model.T = 5.0;       # duration in years 
zero_coupon_test_model.Vₚ = 1000.0;   # face value
zero_coupon_test_model.λ = 1.0;       # payments per year

# Compute -
VB_zc_example = price(zero_coupon_test_model);

In [5]:
# Display -
println("Calculated bill price: $(round(VB_zc_example, digits=0, base=10)) USD. The IEOR price: 822 USD")

Calculated bill price: 822.0 USD. The IEOR price: 822 USD


#### b) Code Check: T = 30 yr Semiannual Coupon Treasury Bond Price

Let's check the ``price`` calculation for a T = 30 year bond taken from Example 14.2 of [Bodie, Kane, Marcus. Investments, 10th ISBN-10: 9780077861674](https://www.amazon.com/Investments-10th-Zvi-Bodie/dp/0077861671)

In [6]:
# build a T = 30 coupon bond model for the test calculation -
coupon_test_model = MultipleCouponBondModel()
coupon_test_model.c̄ = 0.08;    # coupon rate 
coupon_test_model.T = 30.0;    # 30.0 year bond term
coupon_test_model.Vₚ = 1000.0; # face value of the bond
coupon_test_model.r̄ = 0.10;    # interest rate
coupon_test_model.λ = 2.0;     # number of payments per year

# Compute the price of the bond -
VB_30_example = price(coupon_test_model);

In [7]:
# Display -
println("Calculated GT30 bond price: $(round(VB_30_example, digits=2, base=10)) USD. BKM (Example 14.2) price: 810.71 USD");

Calculated GT30 bond price: 810.71 USD. BKM (Example 14.2) price: 810.71 USD


## Bond Pricing Calculations

### a) Compute the price of a T = 30-year treasury bond
Let's use the publicly available data to compute the price of a 30-year bond sold at the most recent auction; current the 08/11/22 auction. 

#### Parameters
A T = 30-year bond was auctioned off on Thursday, August 11, 2022, with a settlement day of Monday, August 15, 2022 (this gives the term date: 08/15/2052). Compute the _fair price_ of the T = 30 year Treasury Bond (GT30) with par value of $V_{p}$ = 1000 USD assuming an annual coupon rate of $\bar{c}$ = 3.0%; coupon payments are made semi-annually (simple interest).

[Recent market interest rates are available for GT20 and GT30](./data/Treasury-HistoricalData-09-09-22.csv) from the Federal Reserve. 

In [16]:
# build a T = 30 coupon bond model for the auction values
GT30_auction_model = MultipleCouponBondModel()
GT30_auction_model.c̄ = 0.03;     # coupon rate 
GT30_auction_model.T = 30.0;     # 30.0 year bond term
GT30_auction_model.Vₚ = 1000.0;  # face value of the bond
GT30_auction_model.λ = 2.0;      # number of payments per year

# what is the market interest rate for 08/11/2022 (or as close as we can get to it)?
# Look up from historical treasury data (in the data folder)
GT30_auction_model.r̄ = 0.0306;   # interest rate from 08/12/2022 (from FedReserve)

# Compute the price of the bond at auction -
GT30_computed_price_at_auction = price(GT30_auction_model)*(100/1000) # prices are reported as fraction of par

98.82765206189065

### b) Compare the calculated versus quoted price for bond
The latest price quote from the Wall Street Journal (WSJ) for T = 30-year bonds reported on `09/16/2022` is included in the [data folder.](./data/WSJ-GT30-Quotes.pdf) The most recent interest rate data can be found at: https://www.federalreserve.gov/releases/h15/

In [17]:
# build a T = 30 coupon bond model for the auction values
GT30_current_model = MultipleCouponBondModel()
GT30_current_model.c̄ = 0.03;     # coupon rate 
GT30_current_model.T = 30.0;     # 30.0 year bond term
GT30_current_model.Vₚ = 1000.0;  # face value of the bond
GT30_current_model.λ = 2.0;      # number of payments per year

# what is the market interest rate for 08/11/2022 (or as close as we can get to it)?
# Look up from historical treasury data (in the data folder)
GT30_current_model.r̄ = 0.0352;   # interest rate quote from 09/19/2022 (from FedReserve)

# Compute the price of the bond at auction -
GT30_computed_price_now = price(GT30_current_model)*(100/1000)

90.41329982746937

#### What is percentage error $\epsilon$ between the estimated and quoted price?
Let's define the percentage error $\epsilon$ as:

$$\epsilon \equiv 100\times\left(\frac{\hat{V}_{B} - V_{B}}{V_{B}}\right)$$

where $\hat{V}_{B}$ denotes the estimated price, and $V_{B}$ denotes the quoted price.

In [10]:
GT30_quoted_price = 90.134; # Bid price WSJ 09/16/2022

In [18]:
# compute percentage diff -
Δ_quote = (GT30_computed_price_now - GT30_quoted_price)/(GT30_quoted_price)*100;

# display -
println("Percentage error in computed bond price: $(round(Δ_quote, digits=2, base=10))%")

Percentage error in computed bond price: 0.31%


### c) Should we hold or sell our bond?
In a sense, the answer to this question is subjective; different investors have different profit targets dictating this decision. However, we'll assume that a _rational_ investor will _never_ sell at a loss.

#### What is the percentage change between the purchase and the current price?
Let's play out a hypothetical; suppose we purchased the GT30 at auction on `08/11/22`. What is that security worth on `09/16/22`; does it make sense to sell this GT30 on the secondary bond market or hold it?

Define the percentage change in the bond price as:
$$\Delta = 100\times\left(\frac{V^{\star}_{B} - \hat{V}^{\circ}_{B}}{\hat{V}^{\circ}_{B}}\right)$$

where $V^{\star}_{B}$ denotes, the quoted price on `09/16/22` and $\hat{V}^{\circ}_{B}$ represents the estimated purchase price.

__Decision rule__:
* If $\Delta\leq{0}$, the bond price has decreased. Do NOT sell.
* If $\Delta>0$, the bond price has decreased. Sell.

In [12]:
# what is the change in the market price of the bond?
Δ_market = (GT30_computed_price_now - GT30_computed_price_at_auction)/(GT30_computed_price_at_auction)*100

# display -
println("Percentage change in the market price of the bond: $(round(Δ_market, digits=2, base=10))%")

Percentage change in the market price of the bond: NaN%
